# 2D Quantum Simulation
## 1) The simple approach

In this simulation, we want to solve the two-dimensional time-dependent Schrodinger equation in the presence of electromagnetic fields. For simplicity, we will limit the magnetic field to one that is given by a uniform field in the z direction: $\boldsymbol{B}(\boldsymbol{x}) = B \hat{z}$, but will entertain an arbitrary electric scalar potential $V(\boldsymbol{x}, t)$. For future reference, it should be noted that the vector potential for this field configuration is given by $\boldsymbol{A}(\boldsymbol{x}) = -\frac{1}{2} \boldsymbol{x} \times \boldsymbol{B} = \frac{B}{2} (x \hat{y} - y \hat{x})$.

Our first step is to write out the Schrodinger equation in the presence of electromagnetic fields:
$$
\frac{1}{2m} \left(\hat{\boldsymbol{p}} - q\boldsymbol{A}\right)^2 \psi + V\psi = \hat{E} \psi
$$

We make the substitutions $\hat{\boldsymbol{p}} \rightarrow -i \nabla$, $\hat{E} \rightarrow i \partial_t$, and expand the momentum operator:
$$
\frac{1}{2m} \left(-\nabla^2 + iq\nabla\cdot\boldsymbol{A} + 2iq\boldsymbol{A}\cdot\nabla + q^2A^2\right)^2 \psi + V \psi = i\partial_t\psi
$$

If we assume the Coulomb gauge ($\nabla\cdot\boldsymbol{A} = 0$) and substitute our expression for the vector potential, this becomes
$$
\frac{1}{2m} \left(-(\partial_x^2\psi + \partial_y^2\psi) + iqB\left(x\partial_y\psi - y\partial_x\psi\right) + \frac{q^2B^2}{4}(x^2+y^2)\psi \right) + V\psi = i\partial_t\psi
$$

If we further split up the wave function into real/imaginary parts $\psi = R + iI$, then we get two PDE's (one for each component):
$$
\begin{align}
\partial_t R &= \frac{1}{2m} \left(-(\partial_x^2 I + \partial_y^2 I) + qB(x\partial_y R - y\partial_x R) + \frac{q^2B^2}{4}(x^2+y^2)I\right) + VI \\
\partial_t I &= \frac{1}{2m} \left((\partial_x^2 R + \partial_y^2 R) + qB (x\partial_y I - y\partial_x I) - \frac{q^2B^2}{4}(x^2 + y^2)R\right) - VR
\end{align}
$$

These can be further simplified by non-dimensionalization. Consider the transformations
$$
(x, y, t) \rightarrow (x/m, y/m, 2t/m) \\
B \rightarrow (m^2/q) B \\
V \rightarrow (m/2) U
$$
which result in the equations
$$
\begin{align}
\partial_t R &= -(\partial_x^2 I + \partial_y^2 I) + B(x\partial_y R - y\partial_x R) + \Gamma I \\
\partial_t I &= (\partial_x^2 R + \partial_y^2 R) + B (x\partial_y I - y\partial_x I) - \Gamma R
\end{align}
$$
where $\Gamma = \frac{B^2}{4}(x^2 + y^2) + U$ (which can be computed ahead of time for computational efficiency)

It is then a simple matter to discretize these equations and solve for the values of the wave function at the next time step. We shall consider two different discretization schemes; first the Forward Time Centered Space (FTCS):
$$
\partial_t X = (X_{t+1} - X_t)/dt,\qquad \partial_x X = (X_{i+1} - X{i-1})/2\,dx,\qquad \partial_x^2 X = (X_{i+1} - 2X_{i} + X_{i-1})/dx^2;
$$
which is quite simple to implement, but is unfortunately numerically unstable. If we instead use a Centered Time Centered Space (CTCS) scheme:
$$
\partial_t X = (X_{t+1} - X_{t-1})/2\,dt,\qquad \partial_x X = (X_{i+1} - X{i-1})/2\,dx,\qquad \partial_x^2 X = (X_{i+1} - 2X_{i} + X_{i-1})/dx^2;
$$
the stability increases greatly (it is not unconditionally stable, but should be stable for $dt/2m\,dx^2 \ll 1$), but at the small cost of having to keep around data from the previous timestep. In practice, we will use both; the FTCS scheme will be used on the very first time step (since the Schrodinger equation is first order, we only want to specify the initial timestep, and not its derivative), and any further updates will use the CTCS step.

## 2) The fancier approach

The above algorithm is quite simple to implement, but suffers from some drawbacks. First of all, even though the CTCS finite-difference algorithm is more stable than the FTCS one, it is still only _conditionally_ stable, which means that we need to take many small timesteps in order to keep numerical stability. If we instead had an _unconditionally_ stable algorithm, we could take larger timesteps, and evolve further in time while doing less computation (taking too large of a timestep can still result in increased error, but at least the solution will not explode). There are a couple different ways to do this; the typical way is to use an _implicit_ solution method (like Crank-Nicholson), where at each timestep you set up a set of linear equations which are then solved to give you the next timestep.

However, while doing some literature search, I found [a very cool algorithm](https://aip.scitation.org/doi/pdf/10.1063/1.168483) that actually allows you to solve _analytically_ for the time evolution operator of spatially discretized 2D system, rendering explicit time steps unconditionally stable. We start in pretty much the same place; only instead of taking the Coulomb gauge, we choose one that is more convenient for numerical computations: $\boldsymbol{A} = A(x,y) \hat{x}$, which implies $\boldsymbol{B} = -\partial_y A \hat{z}$ (if $\boldsymbol{B} = B\hat{z}$, then $A = -By$). The Schrodinger equation ends up looking like
$$
\frac{1}{2m} \left(-\nabla^2\psi + iq \partial_x (A \psi) + iq A \partial_x\psi + q^2A^2\psi\right) + V \psi = i\partial_t\psi
$$

We can make further simplifications by non-dimensionalizing the equation. Once again, we make the following substitutions
$$
(x, y, t) \rightarrow (x/m, y/m, 2t/m) \\
A \rightarrow (m/q) A \\
V \rightarrow (m/2) U
$$
The equation simplifies to
$$
\hat{H}\psi = -(\partial_x^2 + \partial_y^2)\psi + i\partial_x(A\psi) + iA\partial_x\psi + \Gamma\psi = i\dot{\psi} \\
\text{where}\quad
\Gamma = A^2 + U
$$
The function $\Gamma$ is independent of time (assuming the potentials are fixed) and therefore can be calculated ahead of time for computational efficiency.

Now, we discretize the system in space using 4th-order finite differences (you can find the coefficients tabulated [here](https://en.wikipedia.org/wiki/Finite_difference_coefficient#Central_finite_difference)). This results in an approximation of (I will suppress time dependence and spatial indices that don't change, so e.g. $\psi_{i,j+1}(t) = \psi_{j+1}$)
$$
\begin{align}
(\hat{H}\psi)_{ij} &\approx -\frac{1}{\delta_x^2} \left( -\frac{1}{12}\psi_{i-2} + \frac{4}{3}\psi_{i-1} - \frac{5}{2}\psi + \frac{4}{3}\psi_{i+1} - \frac{1}{12}\psi_{i+2} \right) - \frac{1}{\delta_y^2} \left( -\frac{1}{12}\psi_{j-2} + \frac{4}{3}\psi_{j-1} - \frac{5}{2}\psi + \frac{4}{3}\psi_{j+1} - \frac{1}{12}\psi_{j+2} \right) \\
&\qquad + i\frac{1}{\delta_x} \left( \frac{1}{12}A_{i-2}\psi_{i-2} - \frac{2}{3}A_{i-1}\psi_{i-1} + \frac{2}{3}A_{i+1}\psi_{i+1} - \frac{1}{12}A_{i+2}\psi_{i+2} \right) + iA\frac{1}{\delta_x} \left( \frac{1}{12}\psi_{i-2} - \frac{2}{3}\psi_{i-1} + \frac{2}{3}\psi_{i+1} - \frac{1}{12}\psi_{i+2} \right) + \Gamma \psi \\
&= \frac{1}{12\delta^2} \left(1 + i\delta (A + A_{i-2})\right) \psi_{i-2} - \frac{4}{3\delta^2} \left(1 + i\frac{\delta}{2} (A + A_{i-1})\right) \psi_{i-1} + \frac{5}{\delta^2} \left(1 + \frac{\delta^2}{5} \Gamma\right) \psi - \frac{4}{3\delta^2} \left(1 - i\frac{\delta}{2}(A + A_{i+1})\right) \psi_{i+1} \\
&\qquad + \frac{1}{12\delta^2} \left(1 - i\delta (A + A_{i+2})\right) \psi_{i+2} + \frac{1}{12\delta^2} \left( \psi_{j-2} - 16\psi_{j-1} -16\psi_{j+1} + \psi_{j+2} \right)
\end{align}
$$
where $(x, y) = (\delta_x i, \delta_y j)$, and in the second step I set $\delta_x = \delta_y = \delta$. Since the coefficients of the $\psi$'s (which, remember, are functions of space) are all independent of time, I'll call the coefficients of the $\psi_{i+n}$ $\alpha^n$, and the coefficients of $\psi_{j+m}$ $\beta^m$ (with $\beta^0 = 0$ to disambiguate from $\alpha^0$), so that we have the deceptively simple relationship
$$
(\hat{H}\psi)_{ij} \approx \sum_{n=-2}^2 \left(\alpha_{ij}^n \psi_{i+n,j} + \beta_{ij}^n \psi_{i,j+n}\right)
$$

Now comes the tricky part. If we think of our quantum state as a superposition of single-particle states that occupy each site of the lattice, then we can represent it as $|\psi\rangle = \sum_{ij} \psi_{ij} c^\dagger_{ij} |0\rangle$, where $\psi_{ij}$ is the scalar wavefunction at each site, and $c^\dagger_{ij}$ is an operator that creates a particle at site $(i,j)$. Clearly, the action of the Hamiltonian on the state is to cause the particles to hop from site to site, which suggests that we rewrite the Hamiltonian in terms of creation/annihilation operators that destroy a particle at one site and then recreate it at another. For example, the term $\alpha_{ij}^1 \psi_{i+1,j} \subset (\hat{H}\psi)_{ij}$ implies that some particle at site $(i+1,j)$ is destroyed and recreated at site $(i,j)$ (moving it to the left by one site), with the matrix element of the transition being $\alpha_{ij}^1$, so it corresponds to a term in the Hamiltonian like $\alpha_{ij}^1 c^\dagger_{ij}c_{i+1,j} \subset \hat{H}$. A similar logic holds for the other terms in the series, so that we can represent the finite-difference Hamiltonian operator as
$$
\hat{H} \approx \sum_{ij} \sum_{n=-2}^{2} \left( \alpha_{ij}^n c^\dagger_{ij} c_{i+n,j} + \beta_{ij}^n c^\dagger_{ij} c_{i,j+n} \right)
$$
There is one curious structural element of the $\alpha$ coefficients, which is that their conjugates are related to each other via $(\alpha^n_{ij})^\dagger = \alpha^{-n}_{i+n,j}$ (technically the $\beta$'s too, but they are simply constants). Because the expression for $\hat{H}$ sums over all $(i,j,n)$ tuples, we are free to reindex terms; if we use this property of the $\alpha$'s, we can turn all terms with $n < 0$ into $n > 0$ terms:
$$
\hat{H} \approx \sum_{ij} \sum_{n=1}^2 \left[ \left(\alpha_{ij}^n c^\dagger_{ij} c_{i+n,j} + (\alpha_{ij}^n)^* c^\dagger_{i+n,j} c_{ij} \right) + \left( \beta_{ij}^n c^\dagger_{ij} c_{i,j+n} + (\beta_{ij}^n)^* c^\dagger_{i,j+n} c_{ij} \right) + \alpha_{ij}^0 \right]
$$

Now, what we would like to do is plug this expression into the time evolution operator $U(t) = e^{-\hat{H}t}$ and reduce it to something simple. Unfortunately, $\hat{H}$ is composed of many different operators, and they don't all commute with each other, so it is not immediately obvious what to do now. However, if we think about this from a physical point of view, we can see that groups of operators corresponding to non-intersecting paths on the lattice will commute with each other. As a nearly trivial example, consider the pair of operators $\mathcal{O}_1 = c^\dagger_{1,j}c_{2,j}$ and $\mathcal{O}_2 = c^\dagger_{100,j}c_{101,j}$, both corresponding to leftward hops, but on opposite sides of the grid. There is no way in which $\mathcal{O}_1 \mathcal{O}_2 |\psi\rangle$ is any different from $\mathcal{O}_2 \mathcal{O}_1 |\psi\rangle$, because moving particles around on opposite sides of the lattice is invariant to the order of operations, so these operators clearly commute with each other:

<center><img src="./assets/distant_commuting.png"></center>

A more nuanced example would be the operators $\mathcal{O}_1 = c^\dagger_{3,j} c_{1,j}$ and $\mathcal{O}_2 = c^\dagger_{4,j} c_{2,j}$, both corresponding to rightward jumps of 2 sites. However, even though the effect of these operators is localized in approximately the same area, the order in which they are applied to the state does not matter because the hops pass through completely different sites:

<center><img src="./assets/nearby_commuting.png"></center>

The previous two examples showed cases where hop operators DO commute, but what about where they DON'T commute? It's pretty simple to imagine an example like that; let's take the two operators $\mathcal{O}_1 = c^\dagger_{3,j}c_{1,j}$ and $\mathcal{O}_2 = c^\dagger_{5,j}c_{3,j}$, both rightward hops by 2 sites, like the above example. However, these ones are "laced" slightly differently, resulting in their non-commutivity. In particlular, if we assume we have a particle located at site $(1,j)$, then $\mathcal{O}_1 \mathcal{O}_2 |1,j\rangle = |3,j\rangle$, while $\mathcal{O}_2 \mathcal{O}_1 |1,j\rangle = |5,j\rangle$, which are clearly different states!

<center><img src="./assets/nearby_noncommuting.png"></center>

Armed with this knowledge, we can partition the Hamiltonian into groups of operators $\{\hat{H}_n\}$ that commute with each other, but do not commute with other groups. So that when we approximate the time evolution step $U(t) = e^{-i\hat{H}t} \approx \prod_n e^{i\hat{H}_m t}$, we can evaluate each one of these exponential factors analytically, so we don't have to worry about about taking small timesteps to keep unitarity! There are probably multiple ways to slice this, but I will stick with the scheme described in the original paper, which breaks up the operators first into groups by the magnitude of the hop (i.e. 0, 1, and 2 step hops), then breaks each one of these groups into multiple different "lacings" that won't interfere with each other. If I define
$$
\hat{H}^n_{\alpha;ij} = \left(\alpha_{ij}^n c^\dagger_{ij} c_{i+n,j} + (\alpha_{ij}^n)^* c^\dagger_{i+n,j} c_{ij} \right), \qquad
\hat{H}^n_{\beta;ij} = \left(\beta_{ij}^n c^\dagger_{ij} c_{i,j+n} + (\beta_{ij}^n)^* c^\dagger_{i,j+n} c_{ij} \right)
$$

Then the Hamiltonian can be split into nine groups:
$$
\begin{align}
\hat{H}_1 &= \sum_{i\in X_1} \sum_j \hat{H}^2_{\alpha;ij};\qquad X_1 = \{4i, 4i+1 | i=0,1,2,3,...\} \\
\hat{H}_2 &= \sum_{i\in X_2} \sum_j \hat{H}^2_{\alpha;ij};\qquad X_2 = \{4i+2, 4i+3 | i=0,1,2,3,...\} \\
\hat{H}_3 &= \sum_{i\in X_3} \sum_j \hat{H}^1_{\alpha;ij};\qquad X_3 = \{2i | i=0,1,2,3,...\} \\
\hat{H}_4 &= \sum_{i\in X_4} \sum_j \hat{H}^1_{\alpha;ij};\qquad X_4 = \{2i+1 | i=0,1,2,3,...\} \\
\hat{H}_5 &= \sum_{i} \sum_{j\in X_5} \hat{H}^2_{\beta;ij};\qquad X_5 = \{4i, 4i+1 | i=0,1,2,3,...\} \\
\hat{H}_6 &= \sum_{i} \sum_{j\in X_6} \hat{H}^2_{\beta;ij};\qquad X_6 = \{4i+2, 4i+3 | i=0,1,2,3,...\} \\
\hat{H}_7 &= \sum_{i} \sum_{j\in X_7} \hat{H}^1_{\beta;ij};\qquad X_7 = \{2i | i=0,1,2,3,...\} \\
\hat{H}_8 &= \sum_{i} \sum_{j\in X_8} \hat{H}^1_{\beta;ij};\qquad X_8 = \{2i+1 | i=0,1,2,3,...\} \\
\hat{H}_9 &= \frac{1}{2} \sum_{ij} \hat{H}^0_{\alpha;ij} = \sum_{ij} \alpha^0
\end{align}
$$

Thinking about each operator in terms of the hopping operation we detailed above, it should be pretty clear that each of the individual terms within any of the $\hat{H}_n$ commutes with each other, and each conjugate pair of operators forms a two-state subsystem whose matrix exponentiation can be solved analytically. Consider two states $|\ell, k\rangle$ and $|\ell^\prime, k^\prime\rangle$ which form a complete basis for the state space. I.e. quite literally
$$
|\ell,k\rangle = \begin{pmatrix} 1 \\ 0 \end{pmatrix} \qquad\text{and}\qquad
|\ell^\prime,k\rangle = \begin{pmatrix} 0 \\ 1 \end{pmatrix}
$$
The hopping operators can therefore be represented as the following 2x2 matrices (try them out on the above states and you'll see that they work)
$$
c^\dagger_{\ell k} c_{\ell^\prime k^\prime} = \begin{pmatrix} 0 & 1 \\ 0 & 0 \end{pmatrix} \qquad
c^\dagger_{\ell^\prime k^\prime} c_{\ell k} = \begin{pmatrix} 0 & 0 \\ 1 & 0 \end{pmatrix} \qquad
c^\dagger_{\ell k} c_{\ell k} = \begin{pmatrix} 1 & 0 \\ 0 & 0 \end{pmatrix} \qquad
c^\dagger_{\ell^\prime k^\prime} c_{\ell^\prime k^\prime} = \begin{pmatrix} 0 & 0 \\ 0 & 1 \end{pmatrix} \qquad
$$

The matrix exponential is easily calculated by performing an eigendecomposition:
$$
\gamma_{\ell k; \ell^\prime k^\prime} = \alpha c^\dagger_{\ell k} c_{\ell^\prime k^\prime} + \alpha^* c^\dagger_{\ell^\prime k^\prime} c_{\ell k}
= \begin{pmatrix} 0 & \alpha \\ \alpha^* & 0 \end{pmatrix} = S D S^{-1}
= \begin{pmatrix} -\alpha/|\alpha| & \alpha/|\alpha| \\ 1 & 1 \end{pmatrix} \begin{pmatrix} -|\alpha| & 0 \\ 0 & |\alpha| \end{pmatrix} \frac{1}{2} \begin{pmatrix} -\alpha^*/|\alpha| & 1 \\ \alpha^*/|\alpha| & 1 \end{pmatrix}
$$
such that
$$
\exp(-i\gamma_{\ell k; \ell^\prime k^\prime}t) = S \exp(-iDt) S^{-1}
= \begin{pmatrix} \cos(|\alpha|t) & -i\frac{\alpha}{|\alpha|}\sin(|\alpha|t) \\ -i\frac{\alpha^*}{|\alpha|}\sin(|\alpha|t) & \cos(|\alpha|t) \end{pmatrix}
= \left(c^\dagger_{\ell k} c_{\ell k} + c^\dagger_{\ell^\prime k^\prime} c_{\ell^\prime k^\prime}\right) \cos(|\alpha|t) - i \frac{1}{|\alpha|} \left(\alpha c^\dagger_{\ell k} c_{\ell^\prime k^\prime} + \alpha^* c^\dagger_{\ell^\prime k^\prime} c_{\ell k}\right) \sin(|\alpha|t)
$$
You can calculate the determinant of this operator pretty easily and see that it is 1, so the time evolution induced is manifestly unitary.

Exponentiating the $\hat{H}_1$ operator therefore leads to the following time evolution operator:
$$
\begin{align}
e^{-i\tau\hat{H}_1} &= \exp\left[-i\tau \sum_{i\in X_1} \sum_j \left(\alpha_{ij}^2 c^\dagger_{ij} c_{i+2,j} + (\alpha_{ij}^2)^* c^\dagger_{i+2,j} c_{ij} \right)\right] \\
&= \prod_{i\in X_1} \prod_j \left[\left(c^\dagger_{ij} c_{ij} + c^\dagger_{i+2,j} c_{i+2,j}\right) \cos(|\alpha^2_{ij}|\tau) - \frac{i}{|\alpha^2_{ij}|} \left(\alpha^2_{ij} c^\dagger_{ij} c_{i+2,j} + (\alpha^2_{ij})^* c^\dagger_{i+2,j} c_{ij}\right) \sin(|\alpha^2_{ij}|\tau)\right]
\end{align}
$$

Application of this operator to the wave function leads to the following "update" rules which can be easily codified:

```python
def step_H1(psi: np.ndarray, tau: float):
    for i in X1:
        for j in range(psi.shape[1]):
            c, s = np.cos(abs(a2[i,j])*tau), np.sin(abs(a2[i,j]*tau))
            psi[i,j], psi[i+2,j] = (
                c * psi[i,j] - 1.0j * s * (a2[i,j]/abs(a2[i,j])) * psi[i+2,j]
                c * psi[i+2,j] - 1.0j * s * (np.conj(a2[i,j])/abs(a2[i,j])) * psi[i,j]
            )
```
where `X1` is the set $X_1$, and the array elements `a2[i,j]` correspond to $\alpha^2_{ij}$. When porting this to more performant C++ code, some extra care will have to be taken to separate real/imaginary components, and a bit of pre-computation can be done if we know the value of $\tau$ ahead of time so that we can pre-allocate the values of `c` and `s`. There would then be eight other update routines `step_H2`, `step_H3`, etc, and a single timestep would be performed by executing all nine of these sub-steps (in principle, the order does matter, but the difference accrued due to the order of execution is below the error incurred due to approximating the time step to first order).

And that's all there is to it! The derivation is pretty long and winding, but the eventual code is pretty simplistic; we just move some probability density between sites several times within each timestep, then repeat. I should note that besides the "several substeps" nature of this algorithm, the other striking difference from a typical finite difference routine is that each step of the for loops updates a _pair_ of sites (for the $\hat{H}_1$ operator, $|i,j\rangle$ and $|i+2,j\rangle$) simultaneously rather than a single site. I think this is actually critical to keeping the algorithm stable because you can ensure that the probability mass transferred between each of the sites is conserved, which can be traced back to the fact that the time evolution operators are unitary.